# Inference VITS Model on CPU and Compilation on Neuron
* Conditional Variational Autoencoder with Adversarial Learning for End-to-End Text-to-Speech
    * Git: https://github.com/jaywalnut310/vits.git



In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

import os
import json
import math
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader

import commons
import utils
from data_utils import TextAudioLoader, TextAudioCollate, TextAudioSpeakerLoader, TextAudioSpeakerCollate
from models import SynthesizerTrn
from text.symbols import symbols
from text import text_to_sequence

from scipy.io.wavfile import write


def get_text(text, hps):
    text_norm = text_to_sequence(text, hps.data.text_cleaners)
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)
    return text_norm

## LJ Speech

In [2]:
hps = utils.get_hparams_from_file("./configs/ljs_base.json")

In [3]:
net_g = SynthesizerTrn(
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    **hps.model)
_ = net_g.eval()

_ = utils.load_checkpoint("models/pretrained_ljs.pth", net_g, None)

In [4]:

def play_tts_cpu(stn_tst, net_g):
    with torch.no_grad():
        x_tst = stn_tst.unsqueeze(0)
        x_tst_lengths = torch.LongTensor([stn_tst.size(0)])
        # audio = net_g.infer(x_tst, x_tst_lengths, noise_scale=.667, noise_scale_w=0.8, length_scale=1)[0][0,0].data.cpu().float().numpy()
        audio = net_g(x_tst, x_tst_lengths, noise_scale=.667, noise_scale_w=0.8, length_scale=1)[0][0,0].data.cpu().float().numpy()    
        ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate, normalize=False))
    return audio

stn_tst = get_text("VITS is Awesome!", hps)    
audio = play_tts_cpu(stn_tst, net_g)

In [5]:
stn_tst = get_text("VITS is Awesome!, How are you doing today?", hps)    
audio = play_tts_cpu(stn_tst, net_g)

## Neuron Complie Test

In [6]:
import torch
import torch_neuron
from torch_neuron import analyze_model

In [7]:
# wrapping model
class VITSWrapper(torch.nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, x, x_lengths, noise_scale, noise_scale_w, length_scale):
        return self.model.infer(x, x_lengths, 
                                noise_scale=noise_scale.item(),
                                noise_scale_w=noise_scale_w.item(), 
                                length_scale=length_scale.item())

wrapped_model = VITSWrapper(net_g)

In [8]:
# create data
# stn_tst = get_text("VITS is Awesome!", hps)
# print("stn_tst: ", stn_tst)
stn_tst = torch.randint(low=1, high=10, size=(33,), dtype=torch.int64)
x_tst = stn_tst.unsqueeze(0)
x_tst_lengths = torch.LongTensor([stn_tst.size(0)])

# 스칼라 값을 텐서로 변환
noise_scale = torch.tensor(0.667)
noise_scale_w = torch.tensor(0.8)
length_scale = torch.tensor(1.0)

print("x_tst: ", x_tst)
print("x_tst shape: ", x_tst.shape)
print("x_tst_length shape: ", x_tst_lengths.shape)
print("x_tst_length: ", x_tst_lengths)
    

x_tst:  tensor([[6, 1, 7, 1, 7, 6, 3, 4, 5, 3, 7, 2, 6, 4, 6, 1, 5, 9, 3, 4, 7, 8, 9, 6,
         2, 8, 1, 6, 1, 5, 5, 3, 1]])
x_tst shape:  torch.Size([1, 33])
x_tst_length shape:  torch.Size([1])
x_tst_length:  tensor([33])


In [9]:
# torch_neuron.trace 사용
traced_model = torch_neuron.trace(wrapped_model, 
                                  (x_tst, x_tst_lengths, 
                                   noise_scale, noise_scale_w, length_scale))

/tmp/ipykernel_32149/1292780826.py:9: TracerWarning: Converting a tensor to a Python number might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  noise_scale=noise_scale.item(),
/tmp/ipykernel_32149/1292780826.py:10: TracerWarning: Converting a tensor to a Python number might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  noise_scale_w=noise_scale_w.item(),
/tmp/ipykernel_32149/1292780826.py:11: TracerWarning: Converting a tensor to a Python number might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  length_scale=lengt

...complexity:  7250

DFNLayout 0 broadcast=ax1_14_12=[0:33:1] block= par= free=ax1_14_12=[0:33:1]
layout_profile(block_axes_tripcount=1, parallel_axes_tripcount=1)
	l 6 1:0   [2] [F] ['0']
	l 17 t4  const [1, 33] [F,F] ['0', 'ax1_14_12']
	s 30 cast0:0   [33] [F] ['ax1_14_12']
	s 33 TextEncoder_8/aten_to/Cast:0   [1, 1, 33] [F,F,F] ['0', '0', 'ax1_14_12']



DFNLayout 1 broadcast=i2_7=[0:192:1] block= par=i1_7=[0:33:1] free=i2_7=[0:192:1]
layout_profile(block_axes_tripcount=1, parallel_axes_tripcount=33)
	l 53 cast0:0   [33] [P] ['i1_7']
	l 41 0:0   [1, 33, 192] [F,P,F] ['0', 'i1_7', 'i2_7']
	s 70 TextEncoder_8/Encoder_31/aten_mul_1/mul:0   [192, 33] [F,P] ['i2_7', 'i1_7']



TensorContractLayout 2 block= load_par= store_par=ax2_78_76=[0:33:1] free=ax3_79_76=[0:33:1]
layout_profile(block_axes_tripcount=1, parallel_axes_tripcount=33)
	l 85 cast0:0   [33] [F] ['ax2_78_76']
	l 74 cast0:0   [33] [F] ['ax3_79_76']
	r       |V2 $6833[ax2_78_76,ax3_79_76] = tensor_contract_multiply_add(|V2 $7

10/12/2024 10:28:33 AM ERROR 32428 [neuron-cc]: ***************************************************************
10/12/2024 10:28:33 AM ERROR 32428 [neuron-cc]:  An Internal Compiler Error has occurred
10/12/2024 10:28:33 AM ERROR 32428 [neuron-cc]: ***************************************************************
10/12/2024 10:28:33 AM ERROR 32428 [neuron-cc]: 
10/12/2024 10:28:33 AM ERROR 32428 [neuron-cc]: Error message:  should not have any PF conflict
10/12/2024 10:28:33 AM ERROR 32428 [neuron-cc]: 
10/12/2024 10:28:33 AM ERROR 32428 [neuron-cc]: Error class:    AssertionError
10/12/2024 10:28:33 AM ERROR 32428 [neuron-cc]: Error location: Unknown
10/12/2024 10:28:33 AM ERROR 32428 [neuron-cc]: Command line:   /home/ubuntu/miniconda3/envs/vits-conda-py310/bin/neuron-cc compile /tmp/tmp2wdl6wqb/graph_def.pb --framework TENSORFLOW --pipeline compile SaveTemps --output /tmp/tmp2wdl6wqb/graph_def.neff --io-config '{"inputs": {"0:0": [[1, 33, 192], "float32"], "1:0": [[1], "int64"]}, "out


Compiler status ERROR


INFO:Neuron:Compile command returned: 1
[INFO] Compile command returned: 1
[WARNING] torch.neuron.trace failed on _NeuronGraph$1025; falling back to native python function call
ERROR:Neuron:neuron-cc failed with the following command line call:
/home/ubuntu/miniconda3/envs/vits-conda-py310/bin/neuron-cc compile /tmp/tmp2wdl6wqb/graph_def.pb --framework TENSORFLOW --pipeline compile SaveTemps --output /tmp/tmp2wdl6wqb/graph_def.neff --io-config '{"inputs": {"0:0": [[1, 33, 192], "float32"], "1:0": [[1], "int64"]}, "outputs": ["TextEncoder_8/aten_to/Cast:0", "TextEncoder_8/aten_split/split:0", "TextEncoder_8/aten_split/split:1", "StochasticDurationPredictor_10/aten_mul/mul:0", "StochasticDurationPredictor_10/aten_detach_1/Const:0", "StochasticDurationPredictor_10/aten_detach_2/Const:0"]}' --verbose 35
Traceback (most recent call last):
  File "/home/ubuntu/miniconda3/envs/vits-conda-py310/lib/python3.10/site-packages/torch_neuron/convert.py", line 413, in op_converter
    neuron_function

...

[WARNING] torch.neuron.trace failed on _NeuronGraph$1055; falling back to native python function call
ERROR:Neuron:cannot import name 'builder' from 'google.protobuf.internal' (/home/ubuntu/miniconda3/envs/vits-conda-py310/lib/python3.10/site-packages/google/protobuf/internal/__init__.py)
Traceback (most recent call last):
  File "/home/ubuntu/miniconda3/envs/vits-conda-py310/lib/python3.10/site-packages/torch_neuron/convert.py", line 413, in op_converter
    neuron_function = self.subgraph_compiler(
  File "/home/ubuntu/miniconda3/envs/vits-conda-py310/lib/python3.10/site-packages/torch_neuron/decorators.py", line 266, in trace
    metaneff, neff = compile_from_neff(
  File "/home/ubuntu/miniconda3/envs/vits-conda-py310/lib/python3.10/site-packages/torch_neuron/decorators.py", line 355, in compile_from_neff
    from torch_neuron.proto import metaneff_pb2
  File "/home/ubuntu/miniconda3/envs/vits-conda-py310/lib/python3.10/site-packages/torch_neuron/proto/metaneff_pb2.py", line 5, in <

..

INFO:Neuron:Number of arithmetic operators (post-compilation) before = 2009, compiled = 0, percent compiled = 0.0%
[INFO] Number of arithmetic operators (post-compilation) before = 2009, compiled = 0, percent compiled = 0.0%
INFO:Neuron:The neuron partitioner created 31 sub-graphs
[INFO] The neuron partitioner created 31 sub-graphs
INFO:Neuron:Neuron successfully compiled 0 sub-graphs, Total fused subgraphs = 31, Percent of model sub-graphs successfully compiled = 0.0%
[INFO] Neuron successfully compiled 0 sub-graphs, Total fused subgraphs = 31, Percent of model sub-graphs successfully compiled = 0.0%
INFO:Neuron:Compiled these operators (and operator counts) to Neuron:
[INFO] Compiled these operators (and operator counts) to Neuron:
INFO:Neuron:Not compiled operators (and operator counts) to Neuron:
[INFO] Not compiled operators (and operator counts) to Neuron:
INFO:Neuron: => aten::Int: 219 [supported]
[INFO]  => aten::Int: 219 [supported]
INFO:Neuron: => aten::ScalarImplicit: 3 [sup

RuntimeError: No operations were successfully partitioned and compiled to neuron for this model - aborting trace!

.....................

In [7]:
# 트레이스된 모델 저장
torch.jit.save(traced_model, "traced_vits_model_neuron.pt")

## 모델 추론

In [11]:
# Prepare input
# stn_tst = get_text("VITS is Awesome!", hps)
stn_tst = get_text("Melon is a great fruit in summer for everybody", hps)

x_tst = stn_tst.unsqueeze(0)
x_tst_lengths = torch.LongTensor([stn_tst.size(0)])
# Convert scalar value to tensor
noise_scale = torch.tensor(0.667)
noise_scale_w = torch.tensor(0.8)
length_scale = torch.tensor(1.0)

In [ ]:
# inference
traced_output = traced_model(x_tst, x_tst_lengths, noise_scale, noise_scale_w, length_scale)
print("output shape:", traced_output[0].shape)

ipd.display(ipd.Audio(traced_output, rate=hps.data.sampling_rate, normalize=False))


## 모델 로딩후 추론 테스트

In [1]:
import torch
import torch_neuron
import commons
import utils
from text import text_to_sequence

hps = utils.get_hparams_from_file("./configs/ljs_base.json")

def get_text(text, hps):
    text_norm = text_to_sequence(text, hps.data.text_cleaners)
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)
    return text_norm

def load_neuron_model(model_path):
    device = torch.device("cpu")  # 먼저 CPU에 로드
    model = torch.jit.load(model_path, map_location=device)
    return torch_neuron.DataParallel(model)


# Prepare input
stn_tst = get_text("VITS is Awesome!", hps)

x_tst = stn_tst.unsqueeze(0)
x_tst_lengths = torch.LongTensor([stn_tst.size(0)])
# Convert scalar value to tensor
noise_scale = torch.tensor(0.667)
noise_scale_w = torch.tensor(0.8)
length_scale = torch.tensor(1.0)

In [2]:
# 사용 예시
model_path = "traced_vits_model_neuron.pt"
loaded_neuron_model = load_neuron_model(model_path)

In [ ]:
# inference
traced_output = loaded_neuron_model(x_tst, x_tst_lengths, noise_scale, noise_scale_w, length_scale)
print("output shape:", traced_output[0].shape)


## Neuron Analyzer

In [ ]:
import torch
import torch_neuron
from torch_neuron import analyze_model

analyzer = analyze_model(wrapped_model, (x_tst, x_tst_lengths, noise_scale, noise_scale_w, length_scale))